![Alt Text](Images/apple-google.png)

# Apple-Google Sentiment Analyzer

<span style="display:block; height: 3px; border: 0; border-top: 3px solid #000; margin: 1em 0; padding: 0;" ></span>

**Introduction**

The South by Southwest Conference (SXSW) is one of the largest tech and media conferences in the world, attracting a diverse range of attendees from developers and entrepreneurs to marketers and social media influencers. During this event, companies like Apple and Google showcase their latest products and technologies, and engage with attendees to gather feedback and build buzz. However, with so much activity taking place at SXSW, it can be difficult for companies to keep track of how customers are feeling about their products and how those feelings may be influencing purchasing decisions.

**The Business Problem:** 

As an industry leader, it is important for Apple and Google to understand the sentiment of their customers during the SXSW conference. The problem is that with so much noise on social media, it is challenging to manually analyze and classify customer sentiment towards their products. By ignoring or not properly analyzing customer sentiment, Apple and Google may miss out on valuable opportunities to improve customer satisfaction, make data-driven business decisions, and ultimately increase revenue.

**The Solution:** 

One potential solution is to leverage sentiment analysis to automatically classify and analyze the customer sentiment towards Apple and Google products during the SXSW conference. By using natural language processing and machine learning algorithms, sentiment analysis can help categorize the tweets collected from Twitter as positive, negative or neutral, thus making it easier for Apple and Google to track customer sentiment and respond to issues quickly. This information can be used to improve customer experience and shape the direction of product development for both companies.

**The Data:** 

The data used for this project comes from CrowdFlower via https://data.world/crowdflower/brands-and-product-emotions, which consists of approximately 9,000 tweets collected during the 2011 South by Southwest Conference. Each tweet in the dataset is labeled as negative, neutral or positive. With this dataset, we can use machine learning algorithms to train models that can predict the sentiment of new tweets and gain insights on customer sentiment towards Apple and Google products during the conference.

## Data Preprocessing - Inspection

In [1]:
!ls Data

tweets.csv


In [2]:
import pandas as pd
import numpy as np
import spacy
import re

In [3]:
df = pd.read_csv('Data/tweets.csv', encoding = 'ISO-8859-1')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
df.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,NaN,No emotion toward brand or product


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [6]:
df.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,9092,3291,9093
unique,9065,9,4
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,No emotion toward brand or product
freq,5,946,5389


In [7]:
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

## Data Preprocessing - Cleaning

In [8]:
# Renaminmg my columns
df = df.rename(columns = {'tweet_text': 'Tweets', 'emotion_in_tweet_is_directed_at': 'Product/Brand', 
                         'is_there_an_emotion_directed_at_a_brand_or_product': 'Sentiment'})
df.head()

,Tweets,Product/Brand,Sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [9]:
df['Sentiment'].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [10]:
df['Product/Brand'].unique()

array(['iPhone', 'iPad or iPhone App', 'iPad', 'Google', nan, 'Android',
       'Apple', 'Android App', 'Other Google product or service',
       'Other Apple product or service'], dtype=object)

In [11]:
# Renaming the values in Sentiment column to "Positive, Negative, Neutral & Unclear"
sentiment_map = {'Positive emotion': 'Positive', 'Negative emotion': 'Negative', 
                            'No emotion toward brand or product': 'Neutral', "I can't tell": 'Unclear'}
df['Sentiment'] = df['Sentiment'].map(sentiment_map)

In [12]:
df['Sentiment'].value_counts()

Neutral     5389
Positive    2978
Negative     570
Unclear      156
Name: Sentiment, dtype: int64

In [13]:
df['Product/Brand'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: Product/Brand, dtype: int64

In [14]:
print("Number of missing values in 'Sentiment' column:", df['Sentiment'].isna().sum())

Number of missing values in 'Sentiment' column: 0


In [15]:
# Check for the number of missing values in the 'Tweets' column
print("Number of missing values in 'Tweets' column:", df['Tweets'].isna().sum())

Number of missing values in 'Tweets' column: 1


In [16]:
# Dropping the single missing value in 'Tweets' to avoid errors and inconsistencies during analysis.
df = df.dropna(subset = ['Tweets'])

In [17]:
# Confirm the row has been dropped
print("Number of missing values in 'Tweets' column after dropping:", df['Tweets'].isna().sum())

Number of missing values in 'Tweets' column after dropping: 0


In [18]:
print("Number of missing values in 'Product/Brand' column:" ,df['Product/Brand'].isna().sum())

Number of missing values in 'Product/Brand' column: 5801


In [19]:
df['Product/Brand'].head(10)

0                 iPhone
1     iPad or iPhone App
2                   iPad
3     iPad or iPhone App
4                 Google
5                    NaN
7                Android
8     iPad or iPhone App
9                  Apple
10               Android
Name: Product/Brand, dtype: object

Since we have 5801 missing values in the Product/Brand column, we need to fill in these missing values in order to use this data in our analysis. In order to do this, we are using the spaCy library to extract named entities from the tweets in the dataset. We are specifically looking for company names in the tweets, so we are extracting entities with the label 'ORG', which represents organizations. We then store these company names in a list called 'company_names'. We can then use this list to fill in the missing values in the Product/Brand column. 

To do this, we create a dictionary mapping the company names we found to their corresponding products or brands. We then loop through each row in the dataset and fill in missing values in the Product/Brand column using the dictionary. If a tweet mentions more than one company name, we use the first company name we find. By filling in these missing values, we can use this data to perform our analysis and gain insights into the sentiment of these tweets towards different products and brands.

In [22]:
# Load the small English model
nlp = spacy.load('en_core_web_sm')

# Create a list to store the extracted company names
company_names = []

# Loop through each tweet in the dataset
for tweet in df['Tweets']:
    # Use spaCy to extract the named entities from the tweet
    doc = nlp(tweet)
    for ent in doc.ents:
        # Check if the entity is an organization and add it to the list if it is
        if ent.label_ == 'ORG':
            company_names.append(ent.text)

# Print the first 50 unique company names
print(list(company_names)[:50])

['SXSW', 'SXSW', 'SXSW', 'Fri #SXSW', 'Google', 'SXSW', 'CTIA', 'Apple', 'Android', 'SXSW', 'SXSW', '@Gowalla', '=&', 'iPad', 'iPad', 'DST', 'iPhone', 'SXSW', 'SXSW', 'SXSW', 'Forbes', 'SXSW', 'iPad', 'iPad', 'Gowalla 3.0', 'Android', 'SXSW', 'SXSW', 'iPad', 'RT', 'iPad', 'Android', 'iPad', 'iPad', 'SXSW', 'iPad', 'SXSW', 'iPad', 'iPad', 'RT @mention Google', 'New Social Network Called Circles', 'Apple', 'iPad', '@mention &', 'VatorNews - Google', 'Apple Force', 'HootSuite - HootSuite Mobile', 'SXSW', 'BlackBerry &', 'Android']


In [23]:
# Create a dictionary of company names and corresponding keywords to search for in tweets
company_keywords = {'Apple': ['apple', 'iphone', 'ipad'], 'Google': ['google', 'android']}

# Loop through each row in the dataset and fill in missing values in the 'Product/Brand' column
for i, row in df.iterrows():
    tweet = row['Tweets']
    product_brand = row['Product/Brand']
    if pd.isna(product_brand):
        for company, keywords in company_keywords.items():
            for keyword in keywords:
                if re.search(keyword, tweet, re.IGNORECASE):
                    df.at[i, 'Product/Brand'] = company
                    break
            if not pd.isna(df.at[i, 'Product/Brand']):
                break

The above code creates a dictionary called 'company_keywords' which contains the company names as keys and a list of keywords associated with each company as the corresponding values.

Then, it loops through each row in the dataset and fills in the missing values in the 'Product/Brand' column. It first checks if the 'Product/Brand' value is missing, and if it is, it loops through the 'company_keywords' dictionary to look for keywords associated with each company.

It then searches for the keywords in the tweet text using regular expressions and if a keyword is found, it assigns the corresponding company name to the 'Product/Brand' column of that row in the dataframe. The loop stops once a company name has been assigned or if all the keywords have been checked and no matches are found.

This code helps to fill in missing values in the 'Product/Brand' column based on the keywords associated with each company.

In [25]:
print("Number of missing in 'Product/Brand' column after filling:", df['Product/Brand'].isna().sum())

Number of missing in 'Product/Brand' column after filling: 761


In [29]:
# Dropping the 761 missing values in the Product/Brand column since may result in errors and inconsistencies
df.dropna(subset = ['Product/Brand'], inplace = True)

In [30]:
print("Number of missing in 'Product/Brand' column after dropping:", df['Product/Brand'].isna().sum())

Number of missing in 'Product/Brand' column after dropping: 0


In [31]:
df['Product/Brand'].value_counts()

Apple                              3779
Google                             2352
iPad                                946
iPad or iPhone App                  470
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: Product/Brand, dtype: int64

In [37]:
# Checking tweets with unknown sentiments to see if we could find a pattern to whether the tweet has a 
# Negative, Neutral, or positive Sentiment.

pd.set_option("display.max_colwidth", 300)
df[df['Sentiment']=='Unclear']

,Tweets,Product/Brand,Sentiment
102,ÛÏ@mention &quot;Apple has opened a pop-up store in Austin so the nerds in town for #SXSW can get their new iPads. {link} #wow,Apple,Unclear
237,"Just what America needs. RT @mention Google to Launch Major New Social Network Called Circles, Possibly Today {link} #sxsw",Google,Unclear
341,The queue at the Apple Store in Austin is FOUR blocks long. Crazy stuff! #sxsw,Apple,Unclear
368,Hope it's better than wave RT @mention Buzz is: Google's previewing a social networking platform at #SXSW: {link},Google,Unclear
413,SYD #SXSW crew your iPhone extra juice pods have been procured.,Apple,Unclear
...,...,...,...
9020,It's funny watching a room full of people hold their iPad in the air to take a photo. Like a room full of tablets staring you down. #SXSW,Apple,Unclear
9032,"@mention yeah, we have @mention , Google has nothing on us :) #SXSW",Google,Unclear
9037,"@mention Yes, the Google presentation was not exactly what I was expecting. #sxsw",Google,Unclear
9058,&quot;Do you know what Apple is really good at? Making you feel bad about your Xmas present!&quot; - Seth Meyers on iPad2 #sxsw #doyoureallyneedthat?,Apple,Unclear


In [40]:
# Checking & addressing duplicates
len(df[df.duplicated()])

20

## Data Preprocessing - Feature Engineering

In [38]:
df['Company'] = df['Product/Brand']

# Replace product/brand names with company names
df['Company'].replace({'Apple': 'Apple', 'Google': 'Google', 'iPad': 'Apple', 'iPad or iPhone App': 'Apple',
                        'iPhone': 'Apple', 'Other Apple product or service': 'Apple',
                        'Other Google product or service': 'Google', 'Android': 'Google', 'Android App': 'Google'},
                       inplace=True)

# Print the value counts for the 'Company' column
print(df['Company'].value_counts())

Apple     5527
Google    2804
Name: Company, dtype: int64


In [39]:
df.head()

,Tweets,Product/Brand,Sentiment,Company
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative,Apple
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive,Apple
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive,Apple
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative,Apple
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive,Google
